In [19]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

from flask_cors import CORS

import pandas as pd

import os
import requests
import io

In [20]:
url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in.csv'
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in2.csv'
s=requests.get(url).content
df2=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

df['Dprio']=df['Dprio'].astype('float')
df['Actual Bookings']=df['Actual Bookings'].astype('float')
df['Ramp-up frontier']=df['Ramp-up frontier'].astype('float')
df['Ideal curve (80% LF)']=df['Ideal curve (80% LF)'].astype('float')
df['Ideal curve (100% LF)']=df['Ideal curve (100% LF)'].astype('float')
df['Phase-down frontier']=df['Phase-down frontier'].astype('float')

df2['Dprio']=df2['Dprio'].astype('float')
df2['NumPax']=df2['NumPax'].astype('float')
df2['SpoilageRisk']=df2['SpoilageRisk'].astype('float')
df2['SpillageRisk']=df2['SpillageRisk'].astype('float')
df2['Intensity_full']=df2['Intensity_full'].astype('float')
df2['Intensity_downweighted']=df2['Intensity_downweighted'].astype('float')

df2=df2.sort_values(by=['DepDate','Dprio'])

dep_dates = sorted(df['DepDate'].unique())
dtime = sorted(df['dtime'].unique())

In [21]:
df2['SpoilageRisk']=df2['SpoilageRisk'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:3]))
df2['SpillageRisk']=df2['SpillageRisk'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:3]))
df2['Intensity_full']=df2['Intensity_full'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:3]))
df2['Intensity_downweighted']=df2['Intensity_downweighted'].apply(lambda x: float(str(str(x).split('.')[0])+'.'+str(str(x).split('.')[1])[0:3]))

# Import external stylesheet

In [22]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Layout

In [ ]:
line_columns = ['Actual Bookings', 'Ramp-up frontier', 'Ideal curve (80% LF)', 'Ideal curve (100% LF)', 'Phase-down frontier']
linecolor_dict = dict(zip(line_columns,['black', 'red', 'blue', 'green', '#E0115F']))
linedash_dict = dict(zip(line_columns,['solid', 'dot', 'solid', 'solid', 'dot']))
name_dict = dict(zip(line_columns,['Actual Bookings', 'Ramp-up frontier', 'Ideal curve (80% LF)', 'Ideal curve (100% LF)', 'Phase-down frontier']))

In [ ]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    
    # MAIN HEADER
    html.Div([html.H1(children='Booking Curve Steering System',
    style={
            'textAlign': 'center',
            'color': 'black',
            'margin-bottom': '2em',
            'padding' : '50px' ,
            'backgroundColor' : '#66b3cc',
            'width': '90.23%',
            'display': 'inline-block'
        }),html.Img(src=app.get_asset_url('LuxairGroup_logo.jpg'), 
              style = {'backgroundColor' : '#66b3cc', 
                       'width': '5%', 
                       'height': '6em', 
                       'display': 'inline-block',
                       'margin-left': '0em'
                      # 'margin-right': '1em'
                      })
                       
#               html.Div(style={'backgroundColor' : '#66b3cc',
#             'width': '50%',
#             'display': 'inline-block'})
        ]),
    
    # DROPDOWN MENUES
    
        #DROPDOWN MENU FOR DEPATURE DATE
    html.Div([html.Div([html.Label('Departure Date', style={'textAlign': 'center'}), 
    dcc.Dropdown(id = 'dropdown_depdates',
    options=[{
    'label': i,
    'value': i
    } for i in dep_dates],
    placeholder="Select a departure date",                                 
    value='All Departure Dates')],
    style={'width': '10%', 'display': 'inline-block', 'margin-bottom': '1.25em', 'margin-left': '50em'}),
        
        # DROPDOWN MENU FOR DEPARTURE TIME
    html.Div([html.Label('Departure Time', style={'textAlign': 'center'}),
    dcc.Dropdown(id = 'dropdown_dtime',
    options=[{
    'label': i,
    'value': i
    } for i in dtime],
    placeholder="Select a departure time",                       
    value='All Departure Times')],
    style={'width': '10%', 'display': 'inline-block', 'margin-bottom': '1.25em', 'margin-left': '2.5em'})        
             ]),

#     html.Div([dcc.Checklist(
#     options=[
#         {'label': 'All', 'value': 'All'},
#         {'label': 'Spoilage risk', 'value': 'Spoilage'},
#         {'label': 'Spillage risk', 'value': 'Spillage'}
#     ],
#     values=['MTL', 'SF'])],
#     style={'width': '10%', 'display': 'inline-block', 'margin-bottom': '1.25em', 'top-bottom': '1.25em'}), 
    
    # CONTAINERS
    
        # TABLE CONTAINER
    html.Div([html.Div([html.H3(children='Risk Table',style={
             'textAlign': 'center',
             'color': 'black',
             'margin-bottom': '0.75em',
         }),
        html.Div(id='table-container')], style={'width': '33%', 'display': 'inline-block'}),
    
        # GRAPH CONTAINER                   
    html.Div([html.H3(children='Booking Curves',style={
             'textAlign': 'center',
             'color': 'black',
             'margin-bottom': '0.75em',
         }),
              dcc.Graph(id='graph-container')],
             style={'width': '60%', 'float': 'right', 'display': 'inline-block'})]),
])

In [ ]:
@app.callback(
    # Output('table-container', 'children'),
    Output('graph-container', 'figure'),
    [Input('dropdown_depdates', 'value'),
    Input('dropdown_dtime', 'value')])
def update_graph(value_depdates, value_dtime):

    dff = df.loc[(df['DepDate'] == value_depdates)&(df['dtime'] == value_dtime),:] # update with your own logic    
    
    traces = []
    for i in line_columns:
        traces.append(go.Scatter(
            x=dff['Dprio'].tolist(),
            y=dff[i].tolist(),
            mode = 'lines',
            name = name_dict[i],
            line = dict(
              dash = linedash_dict[i],
              color = linecolor_dict[i],
              width = 2
           )
    ))
   
    return {
    'data': traces,
    'layout': go.Layout(
        xaxis={'title': 'Days prior to departure', 'range': [dff['Dprio'].min(), 365]},
        yaxis={'title': 'Number of bookings', 'range': [0, max([dff['Phase-down frontier'].max(),dff['Actual Bookings'].max()])]},
        height=650)
    }
    
    #return generate_table(dff)

In [ ]:
@app.callback(
    Output('table-container', 'children'),
    [Input('dropdown_depdates', 'value'),
    Input('dropdown_dtime', 'value')])
def update_table(value_depdates, value_dtime):

    dff = df2.loc[(df2['DepDate'] == value_depdates)&(df2['dtime'] == value_dtime),
        ['Dprio','SpoilageRisk','Intensity_downweighted','Intensity_full','SpillageRisk']] # update with your own logic    
    
    rename_dict=dict(zip(dff.columns,
    ['Days prior to departure', 'Spoilage Risk', 'Intensity (80% LF)', 'Intensity (100% LF)', 'Spillage Risk']))
    
    return dash_table.DataTable(
        id = 'booking_table',
        data=dff.to_dict('rows'),
        columns=[{'id': c, 'name': rename_dict[c]} for c in dff.columns],
        style_header={'backgroundColor': 'rgb(30, 30, 30)','color': 'white', 'textAlign': 'center'},

        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(40, 40, 40)',
            'color': 'white',
            'textAlign': 'center'
        },
        {
            'if': {'row_index': 'odd', 'column_id': 'Dprio'},
            'backgroundColor': '#568dba',
            'color': 'black',
            'textAlign': 'center'
        },    
        {
            'if': {'row_index': 'even'},
            'backgroundColor': 'rgb(50, 50, 50)',
            'color': 'white',
            'textAlign': 'center'
        },
        {
            'if': {'row_index': 'even', 'column_id': 'Dprio'},
            'backgroundColor': '#6798c1',
            'color': 'black',
            'textAlign': 'center'
        },     
        {
             'if': {
                 'column_id': 'SpoilageRisk',
                 'filter': 'SpoilageRisk >= num(0.95)',
             },
            'color': 'red',
        },
#         {
#             'if': {
#                  'column_id': 'Intensity_downweighted',
#                  'filter': 'SpoilageRisk >= num(0.95)',
#              },
#             'color': 'red',
#             'textAlign': 'left',
#         },
        {
            'if': {
                 'column_id': 'SpillageRisk',
                 'filter': 'SpillageRisk >= num(0.95)',
             },
            'color': '#E0115F'
        },
#         {
#             'if': {
#                  'column_id': 'Intensity_full',
#                  'filter': 'SpillageRisk >= num(0.95)',
#              },
#             'color': '#E0115F',
#             'textAlign': 'right',
#         },
        ],
        style_cell={
        # all three widths are needed
        'minWidth': '50px', 'width': '50px', 'maxWidth': '50px',
        'whiteSpace': 'normal'
    },
        
    sorting = True,
    sorting_type = "multi",
    filtering = True
          
        )

In [ ]:
if __name__ == '__main__':
    app.run_server(port=1000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:1000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2019 15:52:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 15:52:58] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 15:52:58] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 15:52:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 15:52:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 15:53:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 15:53:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 15:53:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 15:53:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:00:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:00:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:00:12] "POST /_dash-update-component

127.0.0.1 - - [11/Apr/2019 16:32:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 16:32:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.